# 영어 - 독일어 번역 모델 생성 (seq2seq)
---
[참고블로그](https://codlingual.tistory.com/91)

In [9]:
!pip install torchtext==0.4

     |████████████████████████████████| 53 kB 1.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [1]:
# 라이브러리 로딩
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [2]:
!python -m spacy download en

!python -m spacy download de

     |████████████████████████████████| 12.0 MB 5.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9 MB 5.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


## Tokenizer

In [3]:
# 각 언어에 맞는 tokenizer 불러오기

spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [4]:
def tokenize_de(text):
    # 독일어 tokenize해서 단어들을 리스트로 만든 후 reverse 
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]
    
def tokenize_en(text):
    # 영어 tokenize해서 단어들을 리스트로 만들기
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
# Field 선언

#input
SRC  = Field(tokenize = tokenize_de, init_token= '<sos>', eos_token = '<eos>', lower =True)

#output
TRG  = Field(tokenize = tokenize_en, init_token= '<sos>', eos_token = '<eos>', lower =True)

In [7]:
# exts : 어떤 언어 사용할지 명시 (input 언어를 먼저 씀)
# filed = (입력, 출력)

train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC,TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 750kB/s] 


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 232kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 218kB/s]


In [8]:
# 데이터 확인
# 독일 단어는 역순임.
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


## Vocab 

In [9]:
#최소 2번은 등장해야 vocab에 포함

SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [14]:
# Iterator 
batch_size = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), batch_size=batch_size)

## 모델링

### encoder
- 2 layer RNN
- Layer 1: 독일어 토큰의 임베딩을 입력으로 받고 은닉상태 출력
- Layer 2 : Layer1의 은닉상태를 입력으로 받고 새로운 은닉상태 출력
- 각 layer마다 초기 은닉상태 h_0 필요 (0으로 초기화 ?)
- 각 layer마다 context vector 'z'를 출력

In [ ]:
# encoder 



class Encoder(nn.Module):
    """seq2seq의 encoder


    input_dim : input 데이터의 vocab size 
    단어들의 index가 embedding 함수로 넘겨짐

    emb_dim : embedding layer의 차원
    embedding 함수 : one-hot vector를 emb_dim 길이의 dense vector로 변환

    hid_dim : 은닉 상태의 차원 ( = cell state의 차원)

    n_layers : RNN 안의 레이어 개수 (여기선 2개)

    dropout : 사용할 드롭아웃의 양 (오버피팅 방지하는 정규화 방법)


    """
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout = 0.2):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM( emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        
        #src = [src len, batch_size)]
        embedded = self.dropout(self.embedding(src))
        #embeded = [src len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        return hidden, cell




### decoder
- Layer 1 : 직전 time-stamp로 부터 은닉 상태(s)와 cell state르 받고, 이들과 embedded token인 y_t를 입력으로 받아 새로운 은닉상태와 cell state를 만들어냄
- Layer 2 : Layer 2의 은닉 상태(s)와 Layer 2에서 직전 time-stamp의 은닉 상태(s)와 cell state를 입력으로 받아 새로운 은닉 상태와 cell state를 만들어냄
- Decoder Layer1의 `첫 은닉상태(s)와 cell state` = `context vector (z)` = `Encoder Layer 1의 마지막 은닉상태(h)와 cell state`
- Decoder RNN/LSTM의 맨 위 Layer의 은닉 상태를 Linear Layer인 f에 넘겨서 다음 토큰이 무엇일지 예측함

 

In [ ]:
class Decoder(nn.Module) : 
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        # Decoder에서 항상 n directions = 1
        # 따라서 hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]
        
        # input = [1, batch size]
        input = input.unsqueeze(0)
        
        # embedded = [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input))
        
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        # output = [seq len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        # Decoder에서 항상 seq len = n directions = 1 
        # 한 번에 한 토큰씩만 디코딩하므로 seq len = 1
        # 따라서 output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        
        # prediction = [batch size, output dim]
        prediction = self.fc_out(output.squeeze(0))
        
        return prediction, hidden, cell
        
